- 데이터 설정
    
    [http://bit.ly/perch_data](http://bit.ly/perch_data) 에서 데이터를 가져옵시다.

In [8]:
import pandas as pd

# 링크에서 읽어오기
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


- Species 열을 제외한 나머지 5개는 입력 데이터로 사용. Species 열은 타깃 데이터

In [9]:
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

- 사이킷런의 train_test_split() 사용

In [10]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

ModuleNotFoundError: No module named 'sklearn'

- 전처리 진행 ★★★

In [11]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

ModuleNotFoundError: No module named 'sklearn'

- SGDClassifier 선언
    
    이는 2개의 매개변수가 필요하다.
    - loss는 loss function의 종류를 지정 (여기서는 loss=’log’)
    - max_iter은 epoch 횟수 지정 (10)

In [3]:
from sklearn.linear_model import SGDClassifier

# 사이킷런 1.1.0 버전 이하일 경우 'log_loss'를 'log'로 바꿀 것.
sc = SGDClassifier(loss='log', max_iter=10, random_state=42)
sc.fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

ModuleNotFoundError: No module named 'sklearn'

- 점진적 학습을 진행
    
    객체를 다시 만들지 말고 이미 훈련한 모델(sc)를 추가로 더 훈련해 보자. 
    
    partial_fit()을 사용하면 된다.

In [ ]:
sc.partial_fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

NOTE
아직 점수는 낮지만 epoch를 한 번 더 진행하니 정확도가 향상되기는 한다!

- 과대 과소 적합

In [ ]:
import numpy as np

# 사이킷런 1.1 버전에서는 SGDClassifier의 loss 매개변수 중 
# 로지스틱 손실을 의미하는 'log'가 'log_loss'로 바뀐다는 경고가 발생합니다.
# 사이킷런 1.1 이상을 사용하는 경우 loss='log'를 loss='log_loss'로 변경하세요.
sc = SGDClassifier(loss='log', random_state=42)

train_score = []
test_score = []

classes = np.unique(train_target)

- 300번의 epoch 진행.
    
    train과 test의 score는 매 epoch마다 저장하도록 한다.

In [ ]:
for _ in range(0, 300):            # _는 나중에 사용하지 않고 그냥 버리는 값을 넣어두는 용도로 사용하는 파이썬의 특별한 변수임~
    sc.partial_fit(train_scaled, train_target, classes=classes)
    
    train_score.append(sc.score(train_scaled, train_target))
    test_score.append(sc.score(test_scaled, test_target))

- 그래프로 score의 변화를 확인하자

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_score)
plt.plot(test_score)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

- NOTE  
    잘 보이지는 않지만… 약 100 epoch 이후에는 test 및 train dataset이 벌어지기 시작함을 볼 수 있다.
    이 경우 약 100 epoch 만큼 학습하는 것이 적절해 보인다.

- 100 epoch로 학습 진행 후 score 확인

In [ ]:
# 사이킷런 1.1 버전에서는 SGDClassifier의 loss 매개변수 중 
# 로지스틱 손실을 의미하는 'log'가 'log_loss'로 바뀐다는 경고가 발생합니다.
# 사이킷런 1.1 이상을 사용하는 경우 loss='log'를 loss='log_loss'로 변경하세요.
sc = SGDClassifier(loss='log', max_iter=100, tol=None, random_state=42)
sc.fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

- NOTE  

    SGDClassifier는 일정 epoch 동안 성능이 향상되지 않으면 자동으로 학습을 중지한다.
    tol 매개변수에서 향상될 최솟값을 지정할 수 있다. 위 코드는 tol=None으로 지정하여 자동으로 안 멈추고 지정한 epoch 횟수 만큼 학습을 진행 시킨 것이다.

    참고로 SGDClassifier의 loss 매개변수에 대해서 기억할 것이 있다.
    default로 ‘hinge’라는 알고리즘을 가지고 있다. 이는 SVM(Support Vector Machine)이라는 알고리즘을 활용한 것이다.
    SVM은 굉장히 널리 사용되는 알고리즘이라는 것, 그리고 SGDClassifier는 loss 매개변수에 다양한 손실함수를 정의해줄 수 있다는 것을 기억하자.